Конечный ноутбук с получением предсказания

In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import sys
import pickle
import numpy as np#
from tensorflow import keras
from tensorflow.keras import layers as L
from tensorflow.keras import Model


from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
import lightgbm as lgb 
from bayes_opt import BayesianOptimization
from itertools import combinations

#import tqdm
from tqdm.notebook import tqdm
import time
import warnings
warnings.filterwarnings("ignore")



os.environ["CUDA_VISIBLE_DEVICES"] = "0"
pd.set_option('display.max_columns', None)

# здесь лежат функции для обработки данных
sys.path.append('./')

2023-04-06 07:01:28.493936: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [5]:
import matplotlib.pyplot as plt
plt.style.use("seaborn-pastel")

In [93]:
y_test_preds_lstm2=[]

for fold in list(range(1, 8)):
    with open(f"/home/yads/CONDA_ALFA_BATTLE/data/stock/y_test_preds_lstm2_Fold_{fold}.h5", "rb") as f:
        list0 = pickle.load(f)
    y_test_preds_lstm2.append(list0[0])    

y_test_preds_lstm2

[array([0.01724725, 0.11200462, 0.03677724, ..., 0.01039199, 0.00859849,
        0.00678363], dtype=float32),
 array([0.02239473, 0.13347027, 0.03635969, ..., 0.02394324, 0.01683966,
        0.01427176], dtype=float32),
 array([0.02737996, 0.12980588, 0.03068688, ..., 0.01321725, 0.01386643,
        0.01427173], dtype=float32),
 array([0.01954461, 0.15107231, 0.02552255, ..., 0.04102132, 0.02105766,
        0.0184016 ], dtype=float32),
 array([0.0212324 , 0.11813864, 0.02368645, ..., 0.01252398, 0.00988104,
        0.01033873], dtype=float32),
 array([0.02134811, 0.16327615, 0.03534869, ..., 0.04096659, 0.04292868,
        0.03317199], dtype=float32),
 array([0.02424112, 0.16255091, 0.03304302, ..., 0.02955747, 0.02134014,
        0.01728289], dtype=float32)]

In [124]:
# indexes
ids_list = pd.read_csv('/home/yads/CONDA_ALFA_BATTLE/data/stock/test_ids.csv').values

ids = []

for i in ids_list:
    ids.append(int(i[0]))

ids

[3000014,
 3000020,
 3000027,
 3000043,
 3000049,
 3000067,
 3000086,
 3000097,
 3000103,
 3000106,
 3000124,
 3000139,
 3000146,
 3000173,
 3000231,
 3000236,
 3000254,
 3000267,
 3000268,
 3000269,
 3000296,
 3000338,
 3000362,
 3000363,
 3000375,
 3000382,
 3000383,
 3000387,
 3000391,
 3000403,
 3000404,
 3000408,
 3000409,
 3000441,
 3000442,
 3000445,
 3000452,
 3000461,
 3000476,
 3000492,
 3000498,
 3000505,
 3000507,
 3000512,
 3000523,
 3000525,
 3000539,
 3000543,
 3000572,
 3000620,
 3000631,
 3000657,
 3000663,
 3000706,
 3000707,
 3000718,
 3000729,
 3000736,
 3000784,
 3000785,
 3000827,
 3000856,
 3000865,
 3000870,
 3000879,
 3000895,
 3000908,
 3000925,
 3000952,
 3000956,
 3000968,
 3001046,
 3001055,
 3001079,
 3001109,
 3001115,
 3001118,
 3001121,
 3001125,
 3001155,
 3001173,
 3001193,
 3001199,
 3001203,
 3001210,
 3001243,
 3001245,
 3001252,
 3001300,
 3001307,
 3001310,
 3001321,
 3001336,
 3001363,
 3001365,
 3001367,
 3001375,
 3001417,
 3001443,
 3001448,


In [127]:
# prediction for biLSTM teaching models
score = np.zeros(len(y_test_preds_lstm2[0]))

for i in y_test_preds_lstm2:
    score += i / len(y_test_preds_lstm2)
    
submission = pd.DataFrame({
    "id" : ids,
    "score": score
}) 


submission.to_csv("submission1.csv", index=None)

In [94]:
y_test_preds_lgbm2=[]

for fold in list(range(1, 8)):
    with open(f"/home/yads/CONDA_ALFA_BATTLE/data/stock/y_test_preds_lgbm2_fold_{fold}.h5", "rb") as f:
        list0 = pickle.load(f)
    y_test_preds_lgbm2.append(list0[0])    

y_test_preds_lgbm2

[array([0.01658155, 0.14794001, 0.05326802, ..., 0.05392319, 0.01444213,
        0.01168443]),
 array([0.02547747, 0.20497108, 0.03181601, ..., 0.03865378, 0.01503017,
        0.01463949]),
 array([0.02541539, 0.1386578 , 0.03014067, ..., 0.01575683, 0.01841695,
        0.01735063]),
 array([0.02092326, 0.21930934, 0.03849726, ..., 0.0657104 , 0.01693341,
        0.01779602]),
 array([0.01345316, 0.18166949, 0.03259595, ..., 0.02130744, 0.01940957,
        0.02085875]),
 array([0.01918671, 0.12834656, 0.04635683, ..., 0.03255104, 0.052079  ,
        0.04142957]),
 array([0.02336688, 0.16646893, 0.0408277 , ..., 0.03025721, 0.02188931,
        0.01927321])]

In [128]:
# prediction for embedding teaching LGBM
score = np.zeros(len(y_test_preds_lgbm2[0]))

for i in y_test_preds_lgbm2:
    score += i / len(y_test_preds_lgbm2)
    
submission = pd.DataFrame({
    "id" : ids,
    "score": score
}) 


submission.to_csv("submission2.csv", index=None)

In [96]:
y_test_preds_best=[]

for fold in list(range(1, 8)):
    with open(f"/home/yads/CONDA_ALFA_BATTLE/data/stock/y_test_preds_best_fold_{fold}.h5", "rb") as f:
        list0 = pickle.load(f)
    y_test_preds_best.append(list0[0])    

y_test_preds_best

[array([0.01658155, 0.14794001, 0.05326802, ..., 0.05392319, 0.01444213,
        0.01168443]),
 array([0.02547747, 0.20497108, 0.03181601, ..., 0.03865378, 0.01503017,
        0.01463949]),
 array([0.02541539, 0.1386578 , 0.03014067, ..., 0.01575683, 0.01841695,
        0.01735063]),
 array([0.02022219, 0.18202079, 0.03134563, ..., 0.05191847, 0.01888327,
        0.01809628]),
 array([0.01345316, 0.18166949, 0.03259595, ..., 0.02130744, 0.01940957,
        0.02085875]),
 array([0.02026741, 0.14581136, 0.04085276, ..., 0.03675881, 0.04750384,
        0.03730078]),
 array([0.02336688, 0.16646893, 0.0408277 , ..., 0.03025721, 0.02188931,
        0.01927321])]

In [129]:
# prediction for best score
score = np.zeros(len(y_test_preds_best[0]))

for i in y_test_preds_best:
    score += i / len(y_test_preds_best)
    
submission = pd.DataFrame({
    "id" : ids,
    "score": score
}) 


submission.to_csv("submission3.csv", index=None)